In [16]:
import pandas as pd
import numpy as np
import traceback
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_DOWN

In [17]:
# Multiple file sets to compare
files = [
    {
        "eyeMindFile": "data/gazeData_AP04_EyeMind_collected-dataEyeMind_1666277969476_collected-data.csv",
        "tobiiFile": "data/AP04.tsv",
    },
        {
        "eyeMindFile": "data/gazeData_AP7-no_EyeMind_collected-dataEyeMind_1666344061895_collected-data.csv",
        "tobiiFile": "data/AP07.tsv",
    },
        {
        "eyeMindFile": "data/gazeData_AP10-no_EyeMind_collected-dataEyeMind_1666355266698_collected-data.csv",
        "tobiiFile": "data/AP10.tsv",
    },
        {
        "eyeMindFile": "data/gazeData_AP12-withinTab_EyeMind_collected-dataEyeMind_1666360517604_collected-data.csv",
        "tobiiFile": "data/AP12.tsv",
    },
]

In [18]:
# Functions
def check_same_number_of_gaze_events(tobiiData, eyeMindData):
    """
    Check if tobiiData and eyeMindData have the same number of gaze events
    """
    return len(tobiiData) == len(eyeMindData)


def check_same_eyetracker_timestamps(tobiiData, eyeMindData):
    """
    Check if the events in tobiiData and eyeMindData have the same Eyetracker timestamps
    """
    return tobiiData["Eyetracker timestamp"].equals(eyeMindData["Eyetracker timestamp"])


def check_sameXandYCoordinates(eyeMindData_column, tobiiData_column):
    """
    Check if events in tobiiData and eyeMindData have the same x and y coordinates,
    rounding values both up and down for comparison.
    """
    # Apply custom rounding function
    eyeMindData_column_rounded_up = eyeMindData_column.apply(
        lambda x: Decimal(x).quantize(0, ROUND_HALF_UP)
    ).fillna(-1).astype(int)

    eyeMindData_column_rounded_down = eyeMindData_column.apply(
        lambda x: Decimal(x).quantize(0, ROUND_HALF_DOWN)
    ).fillna(-1).astype(int)

    # Convert tobiiData_column to integers with NaNs replaced by -1
    tobiiData_column = tobiiData_column.fillna(-1).astype(int)
    
    # Excluding empty coordinate data points in tobiiData
    mask = tobiiData_column != -1

    # Compare using both rounded up and rounded down values
    comparison_result_up = np.where(eyeMindData_column_rounded_up[mask] != tobiiData_column[mask])
    comparison_result_down = np.where(eyeMindData_column_rounded_down[mask] != tobiiData_column[mask])
    
    # Check if both rounded up and rounded down values are unequal to the corresponding value
    common_indices = np.intersect1d(comparison_result_up, comparison_result_down)
    
    # Return True if all values are approximately equal, otherwise return False
    return common_indices.size == 0

In [19]:
# Loop through the file pairs
for file_pair in files:
    # Import files to compare
    eyeMindFile = file_pair["eyeMindFile"]
    tobiiFile = file_pair["tobiiFile"]
    
    print(f"---- Reading file pair {eyeMindFile} and {tobiiFile}")

    # Reading eyeMindData
    eyeMindData = pd.read_csv(eyeMindFile)
    
    # Reading tobiiData
    tobiiData = pd.read_csv(tobiiFile, sep='\t')

    # Keep only eyetracking data in eyeMindData
    eyeMindData = eyeMindData[eyeMindData["eventSource"] == "eye-tracker"]
    # Reset the index
    eyeMindData.reset_index(drop=True, inplace=True)

    #The common attribute to use for the comparision is eyeMindData(Timestamp)*1000 and tobiiData(Eyetracker timestamp)
    #set eyeMindData(Eyetracker timestamp) = eyeMindData(Timestamp)*1000
    eyeMindData["Eyetracker timestamp"] = eyeMindData["Timestamp"] * 1000
    
    # Now, we can compare "Eyetracker timestamp" in eyeMindData and tobiiData
    # The recording in TobiiData starts earlier, so we need to start the comparison from the first "Eyetracker timestamp" in eyeMindData
    # The recording in eyeMindData ends first, so we need to stop the comparison at the last "Eyetracker timestamp" in eyeMindData

    # Get the first timestamp in eyeMindData
    first_timestamp_eyeMindData = eyeMindData["Eyetracker timestamp"].iloc[0]

    # Get the last timestamp in eyeMindData
    last_timestamp_eyeMindData = eyeMindData["Eyetracker timestamp"].iloc[-1]

    # Filter tobiiData to only include rows where the timestamp is greater than or equal to first_timestamp_eyeMindData
    tobiiData = tobiiData[tobiiData["Eyetracker timestamp"] >= first_timestamp_eyeMindData]

    # Filter tobiiData to only include rows where the timestamp is less than or equal to last_timestamp_eyeMindData
    # After this step TobiiData DataFrame will contain only the rows where the timestamp is within the range of the first and last timestamps in eyeMindData.
    tobiiData = tobiiData[tobiiData["Eyetracker timestamp"] <= last_timestamp_eyeMindData]

    # Reset the index
    tobiiData.reset_index(drop=True, inplace=True)

    # Convert "Eyetracker timestamp" to float64
    tobiiData["Eyetracker timestamp"] = tobiiData["Eyetracker timestamp"].astype('float64')

    # Running the checks
    try:
        check_same_number_of_gaze_events(tobiiData, eyeMindData)
        check_same_eyetracker_timestamps(tobiiData, eyeMindData)
        
        # Mapping of columns to compare
        column_mapping = {
            'leftX': 'Gaze point left X',
            'leftY': 'Gaze point left Y',
            'rightX': 'Gaze point right X',
            'rightY': 'Gaze point right Y'
        }
        
        # Check if the events in tobiiData and eyeMindData have the same x and y coordinates
        for eyeMind_column, tobii_column in column_mapping.items():
            check_sameXandYCoordinates(eyeMindData[eyeMind_column], tobiiData[tobii_column])
        
        print(f"All checks passed for file pair: {eyeMindFile}, {tobiiFile}")
        
    except AssertionError as e:
        print(f"Error in file pair {eyeMindFile}, {tobiiFile} - {str(e)}")
        # Print stack trace in case of an error
        traceback.print_exc()

---- Reading file pair data/gazeData_AP04_EyeMind_collected-dataEyeMind_1666277969476_collected-data.csv and data/AP04.tsv
All checks passed for file pair: data/gazeData_AP04_EyeMind_collected-dataEyeMind_1666277969476_collected-data.csv, data/AP04.tsv
---- Reading file pair data/gazeData_AP7-no_EyeMind_collected-dataEyeMind_1666344061895_collected-data.csv and data/AP07.tsv
All checks passed for file pair: data/gazeData_AP7-no_EyeMind_collected-dataEyeMind_1666344061895_collected-data.csv, data/AP07.tsv
---- Reading file pair data/gazeData_AP10-no_EyeMind_collected-dataEyeMind_1666355266698_collected-data.csv and data/AP10.tsv
All checks passed for file pair: data/gazeData_AP10-no_EyeMind_collected-dataEyeMind_1666355266698_collected-data.csv, data/AP10.tsv
---- Reading file pair data/gazeData_AP12-withinTab_EyeMind_collected-dataEyeMind_1666360517604_collected-data.csv and data/AP12.tsv
All checks passed for file pair: data/gazeData_AP12-withinTab_EyeMind_collected-dataEyeMind_166636